In [1]:
# # Do not forget to check if your notebook log collection plugin is working
from mining_extension import check_logging
check_logging("http://3.249.245.244:9999")

Request successful!


In [2]:
!pip install sklearn

In [3]:
import pandas as pd
features_path = 'data/task2/'
test_features = pd.read_pickle(features_path+'test_features.pkl')
train_features = pd.read_pickle(features_path+'train_features.pkl')
validation_features = pd.read_pickle(features_path+'validation_features.pkl')

train_features.index = range(train_features.shape[0])
validation_features.index = range(validation_features.shape[0])
test_features.index = range(test_features.shape[0])

train_features.fillna(0,inplace=True)
test_features.fillna(0,inplace=True)
validation_features.fillna(0,inplace=True)

print(train_features.shape)
print(validation_features.shape)
print(test_features.shape)

(5833, 32)
(1927, 32)
(1918, 21)


In [4]:
features = test_features.columns
numeric_features = ['cell_number', 'execution_count',
       'linesofcomment', 'linesofcode', 'variable_count', 'function_count',
       'text/plain', 'image/png', 'text/html', 'execute_result',
       'display_data', 'stream', 'error']

In [5]:
y = train_features.primary_label
X = train_features[features]

In [6]:
y_val = validation_features.primary_label
X_val = validation_features[features]

In [7]:
!pip install scikit-learn

In [24]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier

# Bagging (the best model)

In [36]:
bagging = BaggingClassifier(base_estimator=rf_model, max_samples=0.5, n_estimators=50, random_state=500)

bagging.fit(X[numeric_features], y)

/Users/ksenialysanuk/Documents/GitHub/jupyter-notebook-hackathon/hackathon_env/lib/python3.8/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=RandomForestClassifier(), max_samples=0.5,
                  n_estimators=50, random_state=500)

In [41]:
print("The F1-score on the validation set")
print(f1_score(y_val, bagging.predict(X_val[numeric_features]), average='macro'))

The F1-score on the validation set
0.4244508686817028


In [43]:
y_test = bagging.predict(test_features[numeric_features])
y_test

array(['helper_functions', 'load_data', 'data_preprocessing', ...,
       'data_exploration', 'result_visualization', 'helper_functions'],
      dtype=object)

# Below are weaker models for reference

## Baseline

In [22]:
baseline = RandomForestClassifier(n_estimators=90, class_weight='balanced_subsample', random_state=500, criterion='gini') 
baseline.fit(X[numeric_features], y)
f1_score(y_val, baseline.predict(X_val[numeric_features]), average='macro')

0.3964028660874709

## A tuned random forest

In [10]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced', 'balanced_subsample', None]
}

rf_model = RandomForestClassifier()

grid_search = GridSearchCV(
    rf_model,
    param_grid=param_grid,
    cv=5,
    scoring='f1_micro',
    n_jobs=-1, 
)

grid_search.fit(X[numeric_features], y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'balanced_subsample',
                                          None],
                         'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1_micro')

In [19]:
f1_score(y_val, grid_search.predict(X_val[numeric_features]), average='macro')

# 0.40266193974299236

0.40266193974299236

In [13]:
grid_search.best_params_

# {'class_weight': None,
# 'max_depth': 10,
# 'min_samples_leaf': 4,
# 'min_samples_split': 5,
# 'n_estimators': 300}

{'class_weight': None,
 'max_depth': 10,
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 300}